## 네이버 영화 평점 train 데이터 기반으로 다음 사항에 유의하여 감성분석 후 test 데이터를 이용해 최종 감성 예측을 수행하세요. 
* 네이버 영화 평점 데이터는 http://github.com/e9t/nsmc에서 ratings.txt(전체), ratings._train.txt(학습), rating_test.txt(테스트)를 다운로드 
* 데이터 세트 요약 정보

  - 칼럼 : id, document
  - label : the sentiment class of the review (0:negative, 1: positive)
  - ratings.txt : All 200K reviews
  - ratings_test : 50K reviews held out for testing
  - ratings_train : 150K reviews for training
  
* 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
* 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
* 한글 형태소 분석을 통해 형태소 단어로 토큰화(tokenizer 사용자 함수 작성 추천)
  - 한글 형태소 엔진은 SNS 분석에 적합한 Okt 클래스를 이용
  - morphs() 메서드는 입력 인자로 들어온 문장을 형태소 단어 형태로 토큰화해 list 객체 반환 
  - Okt 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2)
* 사이킷런의 TfidVectorizor를 이용, TF-IDF 피처 모델을 생성(10분 이상 소요)
* DT, RF, LR 을 이용하여 감성 분석 Classification 수행
* 3RO 분류 모델별 교차 검증 및 Parameter 최적화를 GridSearchCV 를 이용하여 수행(최적 분류 모델 선정)
  - DT params = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5],'min_samples_leaf':[1,5,8]}
  - RF params = {'n_estimators':[50,100, 200], 'max_depth':[2,3,5], 'min_samples_leaf':[1,5,8]}  
  - LR params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }\
    C는 규제 강도를 조절하는 alpha 값의 역수로 작을 수록 규제 강도가 큼
  - param_grid=params , cv=3 ,scoring='accuracy', verbose=1(학습진행 상황 표시) 
* 학습데이터에 사용된 TfidVectorizer 객체 변수인 tfidf_vect를 이용해 transform()을 테스트 데이터의 document 칼럼에 수행

In [1]:
import pandas as pd

train_df = pd.read_csv('./dataset/nsmc/ratings_train.txt', sep='\t')
train_df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [2]:
train_df['label'].value_counts( )

0    75173
1    74827
Name: label, dtype: int64

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
id          150000 non-null int64
document    149995 non-null object
label       150000 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
test_df = pd.read_csv('./dataset/nsmc/ratings_test.txt', sep='\t')
test_df.head(5)
test_df.label.value_counts()

1    25173
0    24827
Name: label, dtype: int64

In [2]:
import re

# 'document 칼럼에 Null이 일부 존재하므로 이 값은 공백으로 변환'
train_df = train_df.fillna(' ')
# 문자가 아닌 숫자의 경우 단어적인 의미로 부족하므로 
# 정규 표현식을 이용하여 숫자를 공백으로 변경(정규 표현식으로 \d 는 숫자를 의미함.) 
train_df['document'] = train_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )

# 테스트 데이터 셋을 로딩하고 동일하게 Null 및 숫자를 공백으로 변환
test_df = pd.read_csv('./dataset/nsmc/ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply( lambda x : re.sub(r"\d+", " ", x) )


In [4]:
import warnings
warnings.filterwarnings('ignore')
# 한글 형태소 분석을 통해 형태소 단어로 토큰화
# 한글 형태소 엔진은 SNS 분석에 적합한 Twitter 클래스를 이용

from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    # morphs() 메서드는 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [6]:
# 사이킷런의 TfidVectorizor를 이용, TF-IDF 피처 모델을 생성(10분 소요)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# 위에서 만든 tw_tokenizer() 함수를 tokenizer로 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, \
                             max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [15]:
# Logistic Regression 을 이용하여 감성 분석 Classification 수행.
# 로지스틱 회귀의 하이퍼 파라미터 C를 설정
# C는 규제 강도를 조절하는 alpha 값의 역수로 작을 수록 규제 강도가 크며
# Penalty는 규제의 유형을 설정하며 l1 규제와 ㅣ2 규제가 있으며 기본은 l2 임
lr_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
gcv = GridSearchCV(lr_clf , param_grid=params , cv=3 ,scoring='accuracy', verbose=0 )
gcv.fit(tfidf_matrix_train , train_df['label'] )
print(gcv.best_params_ , round(gcv.best_score_,4))


{'C': 3.5} 0.8592


In [16]:
# 테스트 세트를 이용해 최종 감성 분석 예측 수행
# 학습 데이터를 적용한 TfidfVectorizer 를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함.
# 이유는 학습 시 설정된 TfidVectorizer의 피처 개수와 테스트 데이터를 변환할 피처 개수가 같아야 하기 때문임 
# 학습 데이터에 사용된 TfidfVectorizer 객체변수인 tfidf_vect를 이용해 transform()을 테스트 데이터의 document 칼럼에 수행

from sklearn.metrics import accuracy_score
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = gcv.best_estimator_
lr_preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],lr_preds))

Logistic Regression 정확도:  0.86186


In [ ]:
# rf을 이용하여 감성 분석 Classification 수행
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


rf_clf = RandomForestClassifier(random_state=0)
params = {'n_estimators':[50,100, 200], 'max_depth':[2,3,5], 'min_samples_leaf':[1,5,8]}
grid_cv = GridSearchCV(rf_clf, param_grid = params, cv=3, scoring= 'accuracy',verbose=0)
grid_cv.fit(tfidf_matrix_train,train_df.label)
print(grid_cv.best_params_, round(grid_cv.best_score_,4))

from sklearn.metrics import accuracy_score
tfidf_matrix_test = tfidf_vect.transform(test_df.document)
best_estimator = grid_cv.best_estimator_
rf_preds = best_estimator.predict(tfidf_matrix_test)
print('rf 정확도:', accuracy_score(test_df.label,rf_preds))

In [ ]:
# dt를 이용하여 감성 분석 Classification 수행
dt_clf = DecisionTreeClassifier(random_state=0)
params = {'max_depth':[2,3,5,10], 'min_samples_split':[2,3,5],\
              'min_samples_leaf':[1,5,8]}
gcv_dt = GridSearchCV(dt_clf,param_grid = params, cv=3, scoring='accuracy',verbose=0)
gcv_dt.fit(tfidf_matrix_train,train_df.label)
print(gcv_dt.best_params_, round(gcv_dt.best_score_,4))

tfidf_matrix_test = tfidf_vect.transform(test_df.document)
best_estimator = grid_cv.best_estimator_
dt_preds = best_estimator.predict(tfidf_matrix_test)
print('dt 정확도:', accuracy_score(test_df.label,dt_preds))